In [ ]:
!pip3 install pydub

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
!pip install SpeechRecognition
from pydub import AudioSegment
import math
import speech_recognition as sr
from os import listdir
from os.path import isfile, join

In [ ]:
class SplitWavAudioMubin():
    def __init__(self, folder, filename):
        self.folder = folder
        self.filename = filename
        self.filepath = folder + '/' + filename

        self.audio = AudioSegment.from_wav(self.filepath)

    def get_duration(self):
        return self.audio.duration_seconds

    def single_split(self, from_min, to_min, split_filename):
        t1 = from_min * 60 * 1000
        t2 = to_min * 60 * 1000
        split_audio = self.audio[t1:t2]
        split_audio.export(self.folder + '/' + split_filename, format="wav")

    def multiple_split(self, min_per_split):
        # open a file where we will concatenate
        # and store the recognized text
        fh = open(self.filename.split('.')[0]+".txt", "w+")
        total_mins = math.ceil(self.get_duration() / 60)
        # create a speech recognition object
        r = sr.Recognizer()

        for i in range(0, total_mins, min_per_split):
            split_fn = str(i) + '_' + self.filename
            self.single_split(i, i+min_per_split, split_fn)
            # get the name of the newly created chunk
            # in the AUDIO_FILE variable for later use.
            file = self.folder + '/'+split_fn
            # recognize the chunk
            with sr.AudioFile(file) as source:
                # remove this if it is not working
                # correctly.
                #r.adjust_for_ambient_noise(source)
                audio_listened = r.listen(source)

            try:
                # try converting it to text
                rec = r.recognize_google(audio_listened)
                # write the output to the file.
                #fh.write(rec+"["+str(i)+" min]")
                fh.write(rec+"@"+str(i)+"#")

            # catch any errors.
            except sr.UnknownValueError:
                print("Could not understand audio")

            except sr.RequestError as e:
                print("Could not request results. check your internet connection")
                print(str(i) + ' Done')
                if i == total_mins - min_per_split:
                    print('All splited successfully')


In [ ]:
if __name__ == '__main__':
    dataset="AppleWatch8"
    #data_path="dataset/"+dataset
    audio_path="audio/"+dataset
    text_path="text/"+dataset
    folder = audio_path

    audio_data = [f for f in listdir(audio_path) if isfile(join(audio_path, f))]
    for a in audio_data:
        file = a
        split_wav = SplitWavAudioMubin(folder, file)
        split_wav.multiple_split(min_per_split=1)

Could not understand audio
Could not understand audio
